### create array

In [43]:
import numpy as np
import pandas as pd

# Choose frequency, binsize, longitude, latitude
freq = 'W'
binsize = 5
longitude_W = 134  # minimum is 134
longitude_E = 174  # maximum is 174
latitude_S = 10  # minimum is 10
latitude_N = 60  # minimum is 60

# load earthquake data for defined area
data = pd.read_csv('data/Japan_10_60_134_174_1973_2023_V2.csv', index_col=0)
data['Time'] = pd.to_datetime(data.Time)
data = data[(data.Longitude >= longitude_W) & (data.Longitude <= longitude_E) & (data.Latitude >= latitude_S) & (data.Latitude <= latitude_N)]
data.set_index('Time', inplace=True)
df = data.sort_index()

# Bin the longitude and latitude values into 2x2 degree bins
df['Longitude_bin'] = pd.cut(df['Longitude'], bins=np.arange(longitude_W, longitude_E + 1, binsize))  # Change bin size to 2 degrees
df['Latitude_bin'] = pd.cut(df['Latitude'], bins=np.arange(latitude_S, latitude_N + 1, binsize))  # Change bin size to 2 degrees

# Group the data by longitude bin, latitude bin, depth bin, and day, and compute the maximum magnitude within each group
grouped = df.groupby(['Longitude_bin', 'Latitude_bin', pd.Grouper(freq=freq, level="Time")]).agg({'Magnitude': 'max', 'Depth': 'mean'})
grouped = grouped.unstack().fillna(0)

# Reshape the resulting data into a tensor with shape (time, longitude, latitude, 4)
time = len(grouped.columns.levels[1])
longitude = len(grouped.index.levels[0])
latitude = len(grouped.index.levels[1])
tensor = np.zeros((time, longitude, latitude, 4))

for t in range(time):
    tensor[t, :, :, 0] = grouped['Magnitude'].iloc[:, t].values.reshape(longitude, latitude)
    tensor[t, :, :, 3] = grouped['Depth'].iloc[:, t].values.reshape(longitude, latitude)

latitude_values = np.linspace(latitude_S, latitude_N - binsize, latitude) + binsize / 2
longitude_values = np.linspace(longitude_W, longitude_E - binsize, longitude) + binsize / 2

tensor[:, :, :, 1] = np.repeat(latitude_values, longitude).reshape(latitude, longitude).T
tensor[:, :, :, 2] = np.tile(longitude_values, latitude).reshape(latitude, longitude).T

# Rotate dimensions corresponding to 20 and 25, 90 degrees anti-clockwise
tensor = np.transpose(tensor, axes=(0, 2, 1, 3))
tensor = np.flip(tensor, axis=1)

# Print the shape of the tensor
print(tensor.shape)

# Reshape tensor into matrix
matrix = np.reshape(tensor[:,:,:,0], (tensor.shape[0], -1))
print(matrix.shape)


(2609, 10, 8, 4)
(2609, 80)


In [ ]:
import numpy as np
import pandas as pd

# Choose frequency, binsize, longitude, latitude
freq = 'W'
binsize = 5
longitude_W = 134 # minimum is 134
longitude_E = 174 # maximum is 174
latitude_S = 10 # minimum is 10
latitude_N = 60 # minimum is 60

# load earthquake data for defined area
data = pd.read_csv('data/Japan_10_60_134_174_1973_2023_V2.csv', index_col=0)
data['Time'] = pd.to_datetime(data.Time)
data = data[(data.Longitude >= longitude_W) & (data.Longitude <= longitude_E) & (data.Latitude >= latitude_S) & (data.Latitude <= latitude_N)]
data.set_index('Time', inplace=True)
df = data.sort_index()

# Bin the longitude and latitude values into 2x2 degree bins
df['Longitude_bin'] = pd.cut(df['Longitude'], bins=np.arange(longitude_W, longitude_E+1, binsize))  # Change bin size to 2 degrees
df['Latitude_bin'] = pd.cut(df['Latitude'], bins=np.arange(latitude_S, latitude_N+1, binsize))  # Change bin size to 2 degrees

# Group the data by longitude bin, latitude bin, depth bin, and day, and compute the maximum magnitude within each group
grouped = df.groupby(['Longitude_bin', 'Latitude_bin', pd.Grouper(freq=freq, level="Time")]).max()['Magnitude']
grouped = grouped.unstack().fillna(0)

# Reshape the resulting data into a tensor with shape (1, time, depth, longitude, latitude)
time = len(grouped.columns)
longitude = len(grouped.index.levels[0])
latitude = len(grouped.index.levels[1])
tensor = np.zeros((time, longitude, latitude))

for t in range(time):
    tensor[t, :, :] = grouped.iloc[:, t].values.reshape(longitude, latitude)

# Rotate dimensions corresponding to 20 and 25, 90 degrees anti-clockwise
tensor = np.transpose(tensor, axes=(0, 2, 1))
tensor = np.flip(tensor, axis=1)

# Print the shape of the resulting tensor
print(tensor.shape)

# Reshape tensor into matrix
matrix = np.reshape(tensor, (tensor.shape[0], -1))

# Keep only the columns with at least one number bigger than 0
matrix = matrix[:, (matrix > 0).sum(axis=0) >= 0]
balance = 1 - (matrix >= 4.5).flatten().sum() / len(matrix.flatten())
print(matrix.shape, "mean:", matrix[matrix > 0].mean(), balance)

In [ ]:
df

### select active subregion

In [ ]:
df_transformed = pd.DataFrame(matrix).set_index(grouped.T.index)

active_column = df_transformed.sum(axis=0).argmax()
active_sub = df_transformed[active_column]
active_vec = np.reshape(np.array(active_sub), (-1, 1))
# print column number with highest sum
print("column with highest sum of magnitudes:", active_column)

### split the data

In [ ]:
from sklearn.model_selection import train_test_split

# split data in train, val, test

# Activate when only forecasting one region
# matrix = np.reshape(matrix, (matrix.shape[0], matrix.shape[1], 1))

train, val_test = train_test_split(matrix, test_size=.3, shuffle=False, random_state=43)
val, test = train_test_split(val_test, test_size=.5, shuffle=False, random_state=43)

### create sliding window datasets

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, samp_rate, cutoff):

  input_data = data
  target_data = data
  dataset = timeseries_dataset_from_array(input_data, targets=None, sequence_length=seq_length, sampling_rate=samp_rate, sequence_stride=1, shuffle=False, batch_size=len(data))

  return dataset

# Set lookback timewindow
lookback_window = 10
cutoff = 4.5
future_timesteps = 1
timewindow = lookback_window + future_timesteps
samp_rate = 1

train_dataset = dataset_generator(train, timewindow, samp_rate, cutoff)
val_dataset = dataset_generator(val, timewindow, samp_rate, cutoff)
test_dataset = dataset_generator(test, timewindow, samp_rate, cutoff)

# Create train set
for batch in train_dataset:
    X_train = batch[:,:-future_timesteps,:]
    X_train = np.reshape(X_train, (X_train.shape[0], -1))

    y_train = batch[:,-future_timesteps:,:]# >= cutoff
    y_train = np.max(y_train, axis=1)#.astype(int)
    # y_train = np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1])).astype(int)

# y_train = tf.reshape(y_train, shape=[y_train.shape[0], 1, y_train.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_train = tf.cast(tf.reduce_max(y_train, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create validation set
for batch in val_dataset:
    X_val = batch[:,:-future_timesteps,:]
    X_val = np.reshape(X_val, (X_val.shape[0], -1))

    y_val = batch[:,-future_timesteps:,:]# >= cutoff
    y_val = np.max(y_val, axis=1)#.astype(int)
    # y_val = np.reshape(y_val, (y_val.shape[0], 1, y_val.shape[1])).astype(int)

# y_val = tf.reshape(y_val, shape=[y_val.shape[0], 1, y_val.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_val = tf.cast(tf.reduce_max(y_val, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create test set
for batch in test_dataset:
    X_test = batch[:,:-future_timesteps,:]
    X_test = np.reshape(X_test, (X_test.shape[0], -1))

    y_test = batch[:,-future_timesteps:,:]# >= cutoff
    y_test = np.max(y_test, axis=1)#.astype(int)
    # y_test = np.reshape(y_test, (y_test.shape[0], 1, y_test.shape[1])).astype(int)

# y_test = tf.reshape(y_test, shape=[y_test.shape[0], 1, y_test.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_test = tf.cast(tf.reduce_max(y_test, axis=2, keepdims=True) > 0, dtype=tf.int32)

################################# Use for MLP
# Flatten 1 and 2 dimensions of X's for multivariate MLP
# X_train = np.reshape(X_train, (X_train.shape[0],-1))
# X_val = np.reshape(X_val, (X_val.shape[0],-1))
# X_test = np.reshape(X_test, (X_test.shape[0],-1))

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

### multinomial logistic regression using sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor

# Train a multi-output linear regression model on the data
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train, y_train)

# Predict the target variables
y_pred = model.predict(X_val)

In [ ]:
y_pred_array = np.array(y_pred)[:,:,1]
y_pred = np.reshape(y_pred_array, (y_pred_array.shape[1], y_pred_array.shape[0]))

### multinomial logistic regression using keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


# Define the model architecture
model = Sequential()
model.add(Dense(80, activation='sigmoid', input_shape=(X_train.shape[1],)))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train >= cutoff, epochs=10, batch_size=1, validation_data=(X_val, y_val >= cutoff))

# Predict the target variables
y_pred = model.predict(X_val)

true = y_val >= cutoff
pred = y_pred >= .5

model.summary()

### persistant model

In [ ]:
y_val_shifted = np.roll(y_val, -1, axis=0)
true = y_val_shifted[::-1] >= cutoff
pred = y_val[::-1] >= cutoff

### evaluation metrics

In [ ]:
y_pred = model.predict(X_val)
true = y_val >= cutoff
pred = y_pred >= cutoff

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(np.array(true).flatten(), pred.flatten() )
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(np.array(true).flatten(), pred.flatten() )
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(np.array(true).flatten(), pred.flatten() )
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(np.array(true).flatten(), pred.flatten() )
print('F1 score: %f' % f1)

class_names = ['M<4.5', 'M>=4.5']

print(classification_report(np.array(true).flatten(), pred.flatten(), target_names=class_names))

# Calculate the proportion of the majority per row, column combination over all batches
majority_prop = np.mean(val >= cutoff, axis=0)[:]

# Calculate the complement for values lower than 0.5
majority_prop = np.where(majority_prop < 0.5, 1 - majority_prop, majority_prop)
zeroR = majority_prop.mean()

print("zeroR:", round(zeroR,4))

### confustion matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.set(rc={'figure.figsize':(5,5)})
p = sns.heatmap(confusion_matrix(np.array(true).flatten(), pred.flatten()), annot=True, fmt='g')
p.set_xlabel("Predicted")
p.set_ylabel("True")
p.xaxis.set_ticklabels(['M<4.5', 'M>=4.5'], ha="center", va="center")
p.yaxis.set_ticklabels(['M<4.5', 'M>=4.5'], rotation=0, va="center")

### ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# y_pred = scaler.inverse_transform(y_prob)
# y_test = scaler.inverse_transform(y)

# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(np.array(true).flatten(), pred.flatten())
roc_auc = auc(fpr, tpr)

# Plot micro-average ROC curve
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
        ''.format(roc_auc), color='blue', linewidth=1)

plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
# plt.savefig(savefig)
plt.show()

### PR curve

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc, f1_score
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(np.array(true).flatten(), pred.flatten())
auc = auc(recall, precision)

# Calculate the proportion of the majority per row, column combination over all batches
majority_prop = np.mean(test >= cutoff, axis=0)[:]

# Calculate the complement for values lower than 0.5
majority_prop = np.where(majority_prop < 0.5, 1 - majority_prop, majority_prop)
zeroR = majority_prop.mean()

# plot the precision-recall curves
no_skill = (np.array(y_test).flatten() == 1).sum() / len(np.array(y_test).flatten())
plt.plot([0, 1], [no_skill,no_skill], linestyle='--', label='class 1 = %.3f' % (no_skill))
plt.plot(recall, precision, label='PR_curve, AUC = %.3f' % (auc))
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR Curve')
# show the legend
plt.legend()
# show the plot
plt.show()

### plot predicted against true

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Reshape data to grid
y_pred = np.reshape(pred, (y_pred.shape[0], tensor.shape[1], tensor.shape[2]))
true = np.reshape(true, (true.shape[0], tensor.shape[1], tensor.shape[2]))

# Choose timesteps to plot
timestep = 13


# Extract the data for the chosen timesteps from the tensor
data1 = y_pred[timestep, :, :] 
data2 = true[timestep, :, :]

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4))

# Plot the data in each subplot
sns.heatmap(data1, cmap='viridis', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax1)
sns.heatmap(data2, cmap='viridis', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax2)

# Set the plot titles and axis labels
ax1.set_title(f'Predicted {cutoff} magnitude at timestep {timestep}')
ax1.set_xlabel('Longitude bin')
ax1.set_ylabel('Latitude bin')

ax2.set_title(f'True {cutoff} magnitude at timestep {timestep}')
ax2.set_xlabel('Longitude bin')
ax2.set_ylabel('Latitude bin')

# Show the plot
plt.show()